# Get the absolute position of widgets in groups

This notebook demonstrates how to get the absolute position of widgets that are in groups.

When widgets are *grouped*, their `[ x, y ]` position is given relative to the group, not in absolute terms, relative to the mural canvas.  If you need to know where widgets are positioned relative to each other, but some of the widgets are grouped, you cannot simply compare their `[ x, y ]` values.

The sample mural for this notebook is here: [Sample mural](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-09_absolute-position.json)

You can create that mural using the notebook here: [Mural creating notebook](https://github.com/spackows/MURAL-API-Samples/blob/main/notebooks/sample-09_absolute-position.ipynb)

The sample mural for this notebook (before moving the sticky notes) looks like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-09_absolute-position.png" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [1]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [40]:
g_auth_token = ""

## Step 3: Read the widgets from the mural

In this sample, some widgets in the mural are sticky notes and some are shapes - squares (rectangles, really).

In [38]:
import requests
import json

def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return []
    if "value" not in response_json:
        print( "No value returned" )
        return []
    return response_json["value"]

In [29]:
widgets_arr_ungrouped = listWidgets( g_mural_id, g_auth_token )

## Print the widgets

Notice the x-position of the widgets proceeds from smallest to largest, left to right, as expected: A, B, C.

In [32]:
def orderWidgetsByX( widgets_arr ):
    return sorted( widgets_arr, key=lambda x: x["x"] )
    
def printWidgets( widgets_arr ):
    ordered_widgets = orderWidgetsByX( widgets_arr )
    print( "ID".ljust(19) + "TYPE".ljust( 8 ) + "[   x,   y ]".ljust(15) + "TEXT\n" )
    for widget in ordered_widgets:
        widget_id = widget["id"].ljust(19)
        widget_type = widget["type"].ljust( 8 )
        widget_x = str( round( widget["x"] ) ).rjust( 3 )
        widget_y = str( round( widget["y"] ) ).rjust( 3 )
        pos = "[ " + widget_x + ", " + widget_y + " ]"
        txt = widget["text"] if ( "text" in widget ) else ""
        print( widget_id + widget_type + pos.ljust(15) + txt )

In [33]:
printWidgets( widgets_arr_ungrouped )

ID                 TYPE    [   x,   y ]   TEXT

0-1654051281139    shape   [ 105, 173 ]   A
0-1654051291443    shape   [ 404, 142 ]   B
0-1654051302806    shape   [ 559, 166 ]   C


# Step 4: Group B and C

Select both shapes, B and C.  Then right-click and choose "Group" as shown below:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-09_absolute-position.gif" alt="Group widgets" width="50%" />

# Step 5: Read the widgets from the mural again

This time, notice ordering the widgets by their x-value doesn't work anymore.  

And there's a new widget, an `area` ??

The `[ x, y ]` position of widgets B and C are now given relative to the area that was created by grouping the two widgets.

In [36]:
widgets_arr_grouped = listWidgets( g_mural_id, g_auth_token )

In [37]:
printWidgets( widgets_arr_grouped )

ID                 TYPE    [   x,   y ]   TEXT

0-1654051291443    shape   [   0,   0 ]   B
0-1654051281139    shape   [ 105, 173 ]   A
0-1654051302806    shape   [ 155,  24 ]   C
0-1654054454818    area    [ 404, 142 ]   


# Step 6: Get the absolute position of widgets

Looking more closely at `widgets_arr_grouped`, you can see widgets B and C have a `parentId` attribute that is pointing to the area.

We can get the absolute position of widgets B and C by using the position information of that parent widget.

In [44]:
def getWidget( widget_id_in, widgets_arr ):
    for widget in widgets_arr:
        if widget["id"] == widget_id_in:
            return widget
    return None

def getParentOffset( parent_id, widgets_arr ):
    offset = { "x" : 0, "y" : 0 }
    parent_widget = getWidget( parent_id, widgets_arr )
    if None != parent_widget:
        offset["x"] += parent_widget["x"]
        offset["y"] += parent_widget["y"]
        # Now recurse, in case the parent has a parent...
        parent_offset = getParentOffset( parent_widget["parentId"], widgets_arr )
        offset["x"] += parent_offset["x"]
        offset["y"] += parent_offset["y"]
    return offset

def getAbsoluteWidgets( mural_id, auth_token ):
    widgets_arr = listWidgets( mural_id, auth_token )
    if len( widgets_arr ) < 1:
        print( "Failed to find widgets in the mural" )
        return None
    absolute_widgets_arr = []
    for widget in widgets_arr:
        parent_offset = getParentOffset( widget["parentId"], widgets_arr )
        x = widget["x"] + parent_offset["x"]
        y = widget["y"] + parent_offset["y"]
        absolute_widget = { "id" : widget["id"],
                             "x" : x,
                             "y" : y,
                             "type" : widget["type"] 
                          }
        if "text" in widget:
            absolute_widget["text"] = widget["text"]
        absolute_widgets_arr.append( absolute_widget )
    return absolute_widgets_arr

In [45]:
absolute_widgets_arr_grouped = getAbsoluteWidgets( g_mural_id, g_auth_token )

In [46]:
printWidgets( absolute_widgets_arr_grouped )

ID                 TYPE    [   x,   y ]   TEXT

0-1654051281139    shape   [ 105, 173 ]   A
0-1654051291443    shape   [ 404, 142 ]   B
0-1654054454818    area    [ 404, 142 ]   
0-1654051302806    shape   [ 559, 166 ]   C
